# Midterm Project: ETL and Data Warehousing
Madelyn Khoury (mgk5ybb) and Tiara Allard

DS 2002 Spring 2023

## Design and Strategy

We chose to model bank transactions as the core business process of our data warehouse, so we designed a database schema centered around bank transactions. To see the schema we designed for this project, please look at the ReadMe of our GitHub project. 

Our schema stores information about bank transactions, bank accounts and users involved in transactions, transaction dates, and transaction locations. We were unable to find a database/dataset with all this information, so instead we combined data from multiple different data sources. This had the added benefit of allowing us to meet the requirements for importing data from a number of sources.

We combined several dummy/randomly generated datasets to build a complete data warehouse. First, we got bank transaction and account information from a .csv file stored on our local filesystem. Then, we generated user data from an API and linked it to the accounts and transactions. Finally, we imported location information from the Northwind MySQL database to represent regions in which banking transactions might have occurred.

After processing the data and computing useful fields, we formatted it into our fact and dimension tables in the final data warehouse.

## Imports and Helper Functions

In [ ]:
import sys
!{sys.executable} -m pip install openpyxl
!{sys.executable} -m pip install mysql-connector-python
!{sys.executable} -m pip install pymysql
!{sys.executable} -m pip install sqlalchemy
!{sys.executable} -m pip install uszipcode

In [133]:
import datetime
import json
import mysql.connector
import os
import pandas as pd
import pymysql
import random
import requests
from sqlalchemy import create_engine
from uszipcode import SearchEngine, SimpleZipcode

In [3]:
def get_api_response(url, headers, params, response_type):
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
    
    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

    if response_type == 'json':
        # result = json.dumps(response.json(), sort_keys=True, indent=4)
        result = response.json()
    elif response_type == 'dataframe':
        result = pd.json_normalize(response.json())
    else:
        result = "An unhandled error has occurred!"
        
    return result

In [4]:
# this helper function is inspired by part of one of the provided files, 02-Python-MySQL.ipynb
def get_mysql_dataframe(user_id, pwd, host_name, db_name, sql_query):
    dframe = None
    try:
        conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
        sqlEngine = create_engine(conn_str, pool_recycle=3600)
        connection = sqlEngine.connect()
        try:
            dframe = pd.read_sql(sql_query, connection);
        except:
            print("Sequel query was unsuccessful.")
        connection.close()
        return dframe
    except:
        print("Unable to connect to the MySQL database.")
    return None

In [47]:
# this code snippet is modified from: https://www.geeksforgeeks.org/python-program-to-calculate-age-in-year/ 
def calculate_age(birth_date):
    birth_date = datetime.datetime.strptime(birth_date, '%Y-%m-%d').date()
    today = datetime.date.today()
    try:
        birthday = birth_date.replace(year = today.year)
 
    # raised when birth date is February 29 but it's not a leap year
    except ValueError:
        birthday = birth_date.replace(year = today.year,
                  month = birth_date.month + 1, day = 1) # birth date becomes march 1st
 
    if birthday > today:
        return today.year - birth_date.year - 1
    else:
        return today.year - birth_date.year

In [64]:
def get_region(state_abbreviation):
    if state_abbreviation in {"WA", "OR", "CA", "ID", "MT", "NV", "UT", "CO", "WY", "AK"}:
        return "West"
    elif state_abbreviation in {"AZ", "NM", "TX", "OK"}:
        return "Southwest"
    elif state_abbreviation in {"ND", "SD", "NE", "KS", "MN", "IA", "MO", "WI", "IL", "MI", "OH"}:
        return "Midwest"
    elif state_abbreviation in {"ME", "NH", "MA", "CT", "RI", "VT", "NY", "PA", "DE", "MD", "NJ"}:
        return "Northeast"
    else:
        return "Southeast"

In [93]:
def get_zipcode(city, state):
    search = SearchEngine()
    results = search.by_city_and_state(city, state)
    if len(results) > 0:
        return results[0].zipcode
    else:
        return None

In [208]:
def get_day(date_timestamp):
    return date_timestamp.day

In [203]:
def get_month(date_timestamp):
    return date_timestamp.month_name()

In [212]:
def get_year(date_timestamp):
    return date_timestamp.year

In [216]:
def get_wwek_day(date_timestamp):
    return date_timestamp.day_name()

## Loading in Data
In this section, we will be loading in the data from three sources: an API, a local filesystem, and a relational database.

### Importing Data From Local File System

The core bank transaction information that we will use came from a dataset on Kaggle (https://www.kaggle.com/datasets/apoorvwatsky/bank-transaction-data). We downloaded the data in the form of a csv file and will import it from the local filesystem in order to be used in our data warehouse.

In [14]:
bank_info_path = os.path.join(os.getcwd(), 'bank.xlsx')
bank_info = pd.read_excel(bank_info_path)

In [15]:
bank_info

,Account No,DATE,TRANSACTION DETAILS,CHQ.NO.,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,.
0,409000611074',2017-06-29,TRF FROM Indiaforensic SERVICES,NaN,2017-06-29,NaN,1000000.0,1.000000e+06,.
1,409000611074',2017-07-05,TRF FROM Indiaforensic SERVICES,NaN,2017-07-05,NaN,1000000.0,2.000000e+06,.
2,409000611074',2017-07-18,FDRL/INTERNAL FUND TRANSFE,NaN,2017-07-18,NaN,500000.0,2.500000e+06,.
3,409000611074',2017-08-01,TRF FRM Indiaforensic SERVICES,NaN,2017-08-01,NaN,3000000.0,5.500000e+06,.
4,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,NaN,2017-08-16,NaN,500000.0,6.000000e+06,.
...,...,...,...,...,...,...,...,...,...
116196,409000362497',2019-03-05,TRF TO 1196428 Indiaforensic SE,NaN,2019-03-05,117934.30,NaN,-1.901902e+09,.
116197,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901602e+09,.
116198,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901302e+09,.
116199,409000362497',2019-03-05,IMPS 05-03-20194C,NaN,2019-03-05,109868.65,NaN,-1.901412e+09,.


### Importing Data From API

We've chosen to use the `users` endpoint from random-data-api.com, which randomly generates data for a set of users. This will populate the Users table in our data warehouse.

In [ ]:
size = 10 # only get info on 10 users for now
url = "https://random-data-api.com/api/v2/users"
querystring = {"size":size}
headers = None

# Get information from users API endpoint
users = get_api_response(url, headers, querystring, "dataframe")
users

### Importing Data From Relational Database

To get location data that could represent locations in which bank transactions were completed, we've decided to import information about the shipping location of orders from the `orders` table in the Northwind database. In our data warehouse, this information will represent the location in which a customer instigated a banking transaction; perhaps it could represent the location of physical branches of the bank.

In [100]:
# define variables to set up connection to mySQL database
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"

user_id = "ds2002"
pwd = "UVA!1819"
db_name = "northwind"

First we must get the location-related data from the `orders` table.

In [101]:
sql_query = """
    SELECT ship_address, ship_city, ship_state_province, ship_zip_postal_code, ship_country_region from orders;
"""

In [106]:
locations_info = get_mysql_dataframe(user_id, pwd, host_name, db_name, sql_query)

## Transforming/Cleaning Up the Data
In this section, we will be transforming, cleaning, and doing transformations on the data, as well as separating it into several tables that we can easily import into our data warehouse.

### Transforming the Location Data

We got location info from the Northwind database, but we must remove duplicate values so that we have a table of unique locations.

In [107]:
locations_info = locations_info.drop_duplicates()

It appears that the Northwind database didn't store actual zip codes, but instead put 99999 in for every row. So, we will fill in the table with the correct zip code for each city listed. Additionally, we will add another column to the table which will identify the region of the United States that the location is in.

In [ ]:
locations_info["zipcode"] = locations_info.apply(lambda row: get_zipcode(row["ship_city"], row["ship_state_province"]), axis=1)
locations_info.drop(["ship_zip_postal_code"], axis = 1, inplace = True)

In [109]:
locations_info["region"] = locations_info.apply(lambda row: get_region(row["ship_state_province"]), axis=1)
locations_info

,ship_address,ship_city,ship_state_province,ship_country_region,zipcode,region
0,789 27th Street,Las Vegas,NV,USA,89101,West
1,123 4th Street,New York,NY,USA,10001,Northeast
2,123 12th Street,Las Vegas,NV,USA,89101,West
3,123 8th Street,Portland,OR,USA,97201,West
5,789 29th Street,Denver,CO,USA,80202,West
6,123 3rd Street,Los Angelas,CA,USA,90001,West
7,123 6th Street,Milwaukee,WI,USA,53202,Midwest
8,789 28th Street,Memphis,TN,USA,38103,Southeast
10,123 10th Street,Chicago,IL,USA,60601,Midwest
11,123 7th Street,Boise,ID,USA,83702,West


### Transforming the User Data

To transform the user data, all we had to do was drop some columns of the dataframe. As we can see by looking at the columns of users, there is a lot of superfluous information.

In [166]:
users.columns

Index(['id', 'uid', 'password', 'first_name', 'last_name', 'username', 'email',
       'avatar', 'gender', 'phone_number', 'social_insurance_number',
       'date_of_birth', 'employment.title', 'employment.key_skill',
       'address.city', 'address.street_name', 'address.street_address',
       'address.zip_code', 'address.state', 'address.country',
       'address.coordinates.lat', 'address.coordinates.lng',
       'credit_card.cc_number', 'subscription.plan', 'subscription.status',
       'subscription.payment_method', 'subscription.term'],
      dtype='object')

In [167]:
users.drop(['employment.title', 'employment.key_skill', 'uid','avatar', 'social_insurance_number', 'subscription.plan', 'subscription.payment_method', 'subscription.status', 'subscription.term', 'address.city', 'address.street_name', 'address.street_address', 'address.zip_code', 'address.state', 'address.country', 'address.coordinates.lat', 'address.coordinates.lng'], axis = 1, inplace = True)

We will also calculate the age of each user, that way we have calculations stored in our OLAP database and don't have to compute them on the fly.

In [168]:
users["age"] = users.apply(lambda row: calculate_age(row["date_of_birth"]), axis=1)

In [169]:
users

,id,password,first_name,last_name,username,email,gender,phone_number,date_of_birth,credit_card.cc_number,age
0,773,2BMeVKtJdD,Raylene,Dicki,raylene.dicki,raylene.dicki@email.com,Male,+32 (224) 064-0638 x79678,2002-07-31,5345-8389-6737-1721,20
1,2459,VaxkFsD8hI,Theron,Ferry,theron.ferry,theron.ferry@email.com,Bigender,+381 (622) 595-5082 x9802,2004-11-25,4284269101719,18
2,8047,2Fn5Yeksdy,Jan,Dibbert,jan.dibbert,jan.dibbert@email.com,Male,+593 345-030-8023,1971-08-18,4131-9475-7357-2563,51
3,2532,iCLhm2YRfl,Earle,Ledner,earle.ledner,earle.ledner@email.com,Female,+387 570.520.5489 x874,1988-10-17,6771-8943-5757-9382,34
4,3512,Zw3d7XYsik,Jody,Koss,jody.koss,jody.koss@email.com,Female,+966 238-120-1298 x8817,1968-06-13,4637886907497,54
5,1116,cRpwBgV2AM,Candie,Hoppe,candie.hoppe,candie.hoppe@email.com,Agender,+672 (393) 046-4132 x72754,1959-04-26,6771-8935-2871-9752,63
6,1187,klu7HVWDqb,Georgene,Ziemann,georgene.ziemann,georgene.ziemann@email.com,Genderfluid,+1-284 1-303-187-3453,1977-01-14,4387204406265,46
7,3963,Sg7ELGwPVc,Maxwell,Ledner,maxwell.ledner,maxwell.ledner@email.com,Polygender,+1-939 421-345-6776 x836,1963-05-07,4563492048494,59
8,4677,qU6NZVWHaG,Concetta,Grant,concetta.grant,concetta.grant@email.com,Female,+33 (812) 482-5850 x67295,2001-01-24,4750-0443-6312-5461,22
9,8150,BmDFjgcNyO,Aubrey,Hyatt,aubrey.hyatt,aubrey.hyatt@email.com,Agender,+92 1-870-128-2791 x2386,1965-05-20,6771-8940-0336-8255,57


### Transforming the Account Information

Reviewing the bank_info table we made, we can see that it has columns relating not just to a single transaction, but also to the date and account associated with the transaction.

In [113]:
bank_info.columns

Index(['Account No', 'DATE', 'TRANSACTION DETAILS', 'CHQ.NO.', 'VALUE DATE',
       'WITHDRAWAL AMT', 'DEPOSIT AMT', 'BALANCE AMT', '.'],
      dtype='object')

We will separate this data into three tables: a Transactions fact table, an Accounts dimension, and a Date dimension. 
The bank transaction fact table can store the new balance of an account after a transaction is completed, but we want the bank account table to store current info about each account-- information which is independent of any one transaction. So, we will take the most recent balance for each account and create a "Current Balance" field in the Accounts table.  Since the rows of the spreadsheet were sorted in order of transaction date, then the most recent balance for each account will be the balance in the last-occurring transaction.

In [16]:
account_info = bank_info[["Account No", "BALANCE AMT"]]
account_info

,Account No,BALANCE AMT
0,409000611074',1.000000e+06
1,409000611074',2.000000e+06
2,409000611074',2.500000e+06
3,409000611074',5.500000e+06
4,409000611074',6.000000e+06
...,...,...
116196,409000362497',-1.901902e+09
116197,409000362497',-1.901602e+09
116198,409000362497',-1.901302e+09
116199,409000362497',-1.901412e+09


In [173]:
account_info = account_info.drop_duplicates(subset=["Account No"], keep="last") # keep only the last record for each account
account_info = account_info.reset_index(drop=True) # I'm also going to reset the indices to start from 0

Our schema design includes another field in the accounts table: the ID of the customer who holds this account. We will randomly select users from our users table to act as the "holders" of these accounts. We will also randomly select the type of each account from a list of options.

In [177]:
account_info["User ID"] = account_info.apply(lambda row: random.randint(0, users.shape[0]-1), axis=1)

In [182]:
bank_account_types = ["Checking", "Savings", "Money market (MMA)", "Certificate of deposit (CD)"]
account_info["Account Type"] = account_info.apply(lambda row: bank_account_types[random.randint(0, len(bank_account_types)-1)], axis=1)

In [183]:
account_info

,Account No,BALANCE AMT,User ID,Account Type
0,409000611074',4.622000e+05,1,Certificate of deposit (CD)
1,409000493201',7.435833e+05,0,Money market (MMA)
2,409000425051',-3.567348e+08,3,Checking
3,409000405747',-5.482675e+08,3,Certificate of deposit (CD)
4,409000438611',-5.479193e+08,3,Certificate of deposit (CD)
5,409000493210',-5.463146e+08,8,Checking
6,409000438620',-5.399631e+08,5,Savings
7,1196711',-1.586916e+09,1,Checking
8,1196428',-1.687234e+09,7,Savings
9,409000362497',-1.901417e+09,5,Certificate of deposit (CD)


### Transforming the Date Information

The other aspect of transactions that we want to put in its own dimension table is the date of each transaction. We will generate a date entry for each date that occurs in the table of transaction information, then link the two tables together.

In [196]:
# get a list of unique dates from the table of transaction info
date_info = bank_info.drop_duplicates(subset=["DATE"])
date_info = date_info.reset_index(drop=True) # reset the indices to start from 0
date_info.drop(['Account No', 'TRANSACTION DETAILS', 'CHQ.NO.','VALUE DATE', 'WITHDRAWAL AMT', 'DEPOSIT AMT', 'BALANCE AMT', '.'], axis = 1, inplace = True)
date_info

,DATE
0,2017-06-29
1,2017-07-05
2,2017-07-18
3,2017-08-01
4,2017-08-16
...,...
1289,2017-12-25
1290,2018-04-02
1291,2018-04-29
1292,2018-05-12


In [209]:
date_info["Day"] = date_info.apply(lambda row: get_day(row["DATE"]), axis=1)

In [211]:
date_info["Month"] = date_info.apply(lambda row: get_month(row["DATE"]), axis=1)

In [213]:
date_info["Year"] = date_info.apply(lambda row: get_year(row["DATE"]), axis=1)

In [218]:
date_info["Day of the Week"] = date_info.apply(lambda row: get_week_day(row["DATE"]), axis=1)

NameError: name 'get_week_day' is not defined

In [214]:
date_info

,DATE,Month,Day,Year
0,2017-06-29,June,29,2017
1,2017-07-05,July,5,2017
2,2017-07-18,July,18,2017
3,2017-08-01,August,1,2017
4,2017-08-16,August,16,2017
...,...,...,...,...
1289,2017-12-25,December,25,2017
1290,2018-04-02,April,2,2018
1291,2018-04-29,April,29,2018
1292,2018-05-12,May,12,2018


In [215]:
bank_info

,Account No,DATE,TRANSACTION DETAILS,CHQ.NO.,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,.
0,409000611074',2017-06-29,TRF FROM Indiaforensic SERVICES,NaN,2017-06-29,NaN,1000000.0,1.000000e+06,.
1,409000611074',2017-07-05,TRF FROM Indiaforensic SERVICES,NaN,2017-07-05,NaN,1000000.0,2.000000e+06,.
2,409000611074',2017-07-18,FDRL/INTERNAL FUND TRANSFE,NaN,2017-07-18,NaN,500000.0,2.500000e+06,.
3,409000611074',2017-08-01,TRF FRM Indiaforensic SERVICES,NaN,2017-08-01,NaN,3000000.0,5.500000e+06,.
4,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,NaN,2017-08-16,NaN,500000.0,6.000000e+06,.
...,...,...,...,...,...,...,...,...,...
116196,409000362497',2019-03-05,TRF TO 1196428 Indiaforensic SE,NaN,2019-03-05,117934.30,NaN,-1.901902e+09,.
116197,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901602e+09,.
116198,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901302e+09,.
116199,409000362497',2019-03-05,IMPS 05-03-20194C,NaN,2019-03-05,109868.65,NaN,-1.901412e+09,.


In [201]:
type(date_info["DATE"][0])

pandas._libs.tslibs.timestamps.Timestamp

### Transforming the Transaction Data

Finally, we can clean up the transaction data!

## Creating and Populating the Data Warehouse
In this section, we will create a new SQL database with all the tables specified in our schema. We'll then populate it with the data tables that we created earlier.

## Executing SQL Queries
Finally, in this section, we will execute several SQL queries that aggregate data from at least three of the data tables in our data warehouse.